# Stylegan2 to generate watercolor fruit

This notebook is the outline of how to modify minimally stylegan2 (taken from [the nvidia github repository](https://github.com/NVlabs/stylegan2-ada-pytorch) ) to work on the [USDA Pomological watercolors](https://naldc.nal.usda.gov/usda_pomological_watercolor) the middle resolution thumbnails of which were collected and stored as part of the [github repository](https://github.com/jwilber/USDA_Pomological_Watercolors) of jwilber. 

### Current flow
1. Import the model and the data from the respective github repositories and mount google drive to store the data output files in a way that means you only have to do this step once.
2. The model is modified slightly to work with pytorch versions above 1.9 - to do this one must make sure ninja is installed and also to modify two of the custom scripts to contain references to the later versions - the file names are specifiied before the model training cells.

### TODO
Store the weights on github and clone them for transfer learning further fruit.

Add examples of the fruit generated.

### Remarks about output

Examples of the output can be found in the github repositiory (not yet). The output has a FID score of 58 after only 700 steps - which is much less than the recommended 5000 steps for minimum quality output suggested by the original [paper](https://arxiv.org/pdf/2006.06676.pdf). 

##Importing the model and data, mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch
!git clone https://github.com/jwilber/USDA_Pomological_Watercolors

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 2.20 MiB/s, done.
Resolving deltas: 100% (57/57), done.
Cloning into 'USDA_Pomological_Watercolors'...
remote: Enumerating objects: 7615, done.
remote: Total 7615 (delta 0), reused 0 (delta 0), pack-reused 7615
Receiving objects: 100% (7615/7615), 33.25 MiB | 11.33 MiB/s, done.
Resolving deltas: 100% (14/14), done.


## Manipulating the images using a commandline tool to put them into the format that the model can process.

The model can take only square images that are width/height a power of 2 - I previously used them as 256x256 images, but the training time would be lower at 128x128 - below I installed imagemagick's unix standalone command line tool, made it executable and then did the conversion in size and to pngs (from jpgs) that the model requires. This takes about 10 minutes.

In [ ]:
cd /content/USDA_Pomological_Watercolors/data/images/

/content/USDA_Pomological_Watercolors/data/images


In [ ]:
!mkdir pngs

In [ ]:
!wget https://download.imagemagick.org/ImageMagick/download/binaries/magick

--2022-03-31 13:47:28--  https://download.imagemagick.org/ImageMagick/download/binaries/magick
Resolving download.imagemagick.org (download.imagemagick.org)... 50.251.58.13
Connecting to download.imagemagick.org (download.imagemagick.org)|50.251.58.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27104448 (26M)
Saving to: ‘magick’

magick              100%[===================>]  25.85M   573KB/s    in 40s     

2022-03-31 13:48:09 (663 KB/s) - ‘magick’ saved [27104448/27104448]



In [ ]:
!chmod +x magick

In [ ]:
!./magick '*.jpg[256x256!]' image%04d.png

In [ ]:
!mv *.png /content/USDA_Pomological_Watercolors/data/images/pngs/

In [ ]:
#Here you can adjust the paths to your own specficiations - note down where you are working you will need it later.
!mkdir /content/drive/MyDrive/datascience/poms/pngs

In [ ]:
!mv /content/USDA_Pomological_Watercolors/data/images/pngs/*.png /content/drive/MyDrive/datascience/poms/pngs/ 

## Configuring the dataset and the model

We run our newly minted pngs through their data preparation script to produce a viable zip file of pngs, in our case, in this instance, without labels.

Then we are ready to run the model - with small numbers of steps between savings and for a short-ish length. This will be a fresh training, with the step below to execute for a continuation/resumation of the training at a later time.

This requires us to manipulate two of the configuration files to enable later versions of pytorch beyond what they're expecting, and to install packages to allow them to perform image manipulations every iteration of their model.

In [ ]:
cd /content/stylegan2-ada-pytorch/

/content/stylegan2-ada-pytorch


In [ ]:
!python dataset_tool.py --help

Usage: dataset_tool.py [OPTIONS]

  Convert an image dataset into a dataset
  archive usable with StyleGAN2 ADA PyTorch.

  The input dataset format is guessed from the
  --source argument:

  --source *_lmdb/                    Load LSUN dataset
  --source cifar-10-python.tar.gz     Load CIFAR-10 dataset
  --source train-images-idx3-ubyte.gz Load MNIST dataset
  --source path/                      Recursively load all images from path/
  --source dataset.zip                Recursively load all images from dataset.zip

  Specifying the output format and path:

  --dest /path/to/dir                 Save output files under /path/to/dir
  --dest /path/to/dataset.zip         Save output files into /path/to/dataset.zip

  The output dataset format can be either an
  image folder or an uncompressed zip archive.
  Zip archives makes it easier to move datasets
  around file servers and clusters, and may
  offer better training performance on network
  file systems.

  Images within the dataset

In [ ]:
#You must adjust the path to the pngs you created previously.
!python dataset_tool.py --source /content/drive/MyDrive/datascience/poms/pngs/ --dest /content/drive/MyDrive/datascience/poms/data_pngs.zip

100% 7580/7580 [01:46<00:00, 71.46it/s]


In [ ]:
!python train.py --help

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Usage: train.py [OPTIONS]

  Train a GAN using the techniques described in
  the paper "Training Generative Adversarial
  Networks with Limited Data".

  Examples:

  # Train with custom dataset using 1 GPU.
  python train.py --outdir=~/training-runs --data=~/mydataset.zip --gpus=1

  # Train class-conditional CIFAR-10 using 2 GPUs.
  python train.py --outdir=~/training-runs --data=~/datasets/cifar10.zip \
      --gpus=2 --cfg=cifar --cond=1

  # Transfer learn MetFaces from FFHQ using 4 GPUs.
  python train.py --outdir=~/training-runs --data=~/datasets/metfaces.zip \
      --gpus=4 --cfg=paper1024 --mirror=1 --resume=ffhq1024 --snap=10

  # Reproduce original StyleGAN2 config F.
  python train.py --outdir=~/training-runs --data=~/datasets/ffhq.zip \
      --gpus=8 --cfg=stylegan2 --mirror=1 --aug=noaug

  Base configs (--cfg):
    auto       Automatically select reasonable defaults based on resolution
               and GPU c

In [ ]:
#again, make a new directory to store the output - this is mine.
!mkdir /content/drive/MyDrive/datascience/poms/sgtrain/

One must edit the following files: 


*   /content/stylegan2-ada-pytorch/torch_utils/ops/conv2d_gradfix.py
*   /content/stylegan2-ada-pytorch/torch_utils/ops/grid_sample_gradfix.py

to include '1.10' in lists of versions available in the version checking steps - which can be found on line 53 for conv2d_gradfix.py and line 37 for grid_sample_gradfix.py.

In [ ]:
# these are the packages required - ninja is particularly important for their custom image manipulation libaries. 
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

     |████████████████████████████████| 195 kB 31.3 MB/s 
     |████████████████████████████████| 108 kB 79.5 MB/s 
     |████████████████████████████████| 26.9 MB 1.2 MB/s 


In [ ]:
#To train fresh, remove the --resume path. On the high ram/gpu colab pro machine (p100 gpu) this particular training routine took ~5m/tick, ~15m for a scoring tick and the entire thing lasted 6h49m.
!python train.py --resume /content/drive/MyDrive/datascience/poms/sgtrain/00003-data_pngs-auto1-kimg1000/network-snapshot-000400.pkl --outdir /content/drive/MyDrive/datascience/poms/sgtrain/ --data /content/drive/MyDrive/datascience/poms/data_pngs.zip --gpus 1 --kimg 300 --snap 15


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 15,
  "network_snapshot_ticks": 15,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/datascience/poms/data_pngs.zip",
    "use_labels": false,
    "max_size": 7580,
    "xflip": false,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base":